In [1]:
import requests 
import copy
import json
from wasmer import Instance



In [2]:

s = {
    "baseURL": 'http://192.168.1.140:3000/',
    'id': "5bc59e192817116e84bdd831",
    'currentPoint': False
}

sources = {}
instances = {}

def getFunc(nodes):
    for key, value in nodes.items():
        req = requests.get(s['baseURL']+'pfunction/'+value['id']+'?filter={%22include%22:[{%22relation%22:%22pclass%22,%22scope%22:{%22include%22:[{%22relation%22:%22sources%22}]}}]}')
        func = req.json()
        print(func['pclass']['sources'][0]["fileid"]) # ['pclass']['sources']
        fileid = func['pclass']['sources'][0]["fileid"]
        sources[value['id']] =  requests.get(s['baseURL']+'file/'+fileid+'/source').content
        #print(sources[value['id']])
        instances[value['id']] = Instance(sources[value['id']])
        funcs[func['_id']] = func
        if "graph" in func['data']:
            gr = func['data']['graph']
            if (gr != {}):
                getFunc(gr['n'])

def enrich_graph(gr1):
    gr = copy.deepcopy(gr1)
    for n in gr['n']:
        x = gr['n'][n]
        x['out'] = {}; 
        x['in'] = {};
        x['position'] = {'x': 0, 'y': 0};
    for e in gr['e']:
        if e[0] == 0:
            pass
        else:
            if e[1]  in gr['n'][str(e[0])]['out']:
                gr['n'][str(e[0])]['out'][e[1]].append([e[2], e[3]])
            else:
                gr['n'][str(e[0])]['out'][e[1]] = [[e[2], e[3]]]
        gr['n'][str(e[2])]['in'][e[3]] = [e[0], e[1]]
    add_nodesedges(funcs, gr)
    visitors = [enrich]
    positionNodes(gr, list(gr['n'].keys()), [0], visitors, 0)
    s['currentPoint'] = False
    return gr

    

def add_nodesedges(context, gr):
    added_nodes = {}
    added_edges = []
    step_out = 4000
    step_in = 3000
    for n in gr['n']:
        x = gr['n'][n]
        for idx1, value1 in enumerate(context[x['id']]['data']['gapi']['outputs'], start=1):
            if (idx1 not in x['out'] and x['id'] != s['id']):
                no = { 
                    'i': step_out,  
                    'svg_id': "", 
                    'id': s['id'] + str(step_out), 
                    'in':{1: [x['i'], idx1]}, 
                    'out':{}, 
                    'position': {'x': 0, 'y':0} ,
                    'edges': []
                  }
                typing  = value1 #['type']
                funcs[s['id'] + str(step_out)]  = {"_id": s['id'] + str(step_out),
                   "pclassid":"5c95397d4212cc40afeec914",
                   "data":{"signature":"io", "sources": {"python":"f : f"}, 
                    "graph":{}, "gapi":{"constant": True,"inputs":[{"name":typing['name'],"type": typing['type']}],
                    "name":"io","outputs":[{"name": typing['name'], "type": typing['type']}],"payable": False,
                    "stateMutability":"view","type":"function"},"chainids":["3"]},"tags":["pipeline","ethpm"],
                    "timestamp":""}
                added_nodes[str(step_out)]  = no
                added_edges.append([x['i'], idx1, step_out, 1])
                x['out'][idx1]= [[step_out, 1]]
                step_out= step_out+1
    for n in gr['n']:
        x = gr['n'][n]
        for idx1, value1 in enumerate(context[x['id']]['data']['gapi']['inputs'], start=1):
            if (idx1 not in x['in'] and x['id'] != s['id']):
                no = { 
                    'i': step_in,  
                    'svg_id': "", 
                    'id': s['id'] + str(step_in), 
                    'in': {}, 
                    'out': {1:[[x['i'], idx1]]},
                    'position': {'x': 0, 'y':0} ,
                    'edges': []
                  }
                typing  = value1 #['type']
                funcs[s['id'] + str(step_in)]  = {"_id": s['id'] + str(step_in),
                   "pclassid":"5c95397d4212cc40afeec914",
                   "data":{"signature":"io", "sources": {"python":"f : f"}, 
                    "graph":{}, "gapi":{"constant": True,"inputs":[{"name":typing['name'],"type": typing['type']}],
                    "name":"io","outputs":[{"name": typing['name'],"type": typing['type']}],"payable": False,
                    "stateMutability":"view","type":"function"},"chainids":["3"]},"tags":["pipeline","ethpm"],
                    "timestamp":""}
                added_nodes[str(step_in)]  = no
                added_edges.append([step_in, 1, x['i'], idx1])
                x['in'][idx1]= [step_in, 1]
                step_in= step_in+1
    gr['n'] = {**gr['n'], **added_nodes }
    gr['e'] = gr['e'] + added_edges


def difference(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def inputNodesAreVisited(node, visited):
    for key in node['in']:
        if not (node['in'][key][0] in visited): 
            return False
    return True
    
def inputNodesAreDefined(node, defined, level):
    #print(defined, node['in'])
    if (level == 0):
        return False
    for key in node['in']:
        if node['in'][key][0] in defined:
            return True
        """if not u:
            return False;
        if not u[node['in'][key][0]]:
            return False; """
    return False

def enrich(na):
    if not s['currentPoint']: 
        s['currentPoint'] = {'x': 0, 'y': 1}
    s['currentPoint']['y'] = na['level']
    if (na['row'] == 0):
        s['currentPoint']['x'] = 0
    na['node']['position'] = copy.copy(s['currentPoint'])
    max1  = max( len(na['context'][na['node']['id']]['data']['gapi']['inputs']), 
                  len(na['context'][na['node']['id']]['data']['gapi']['outputs']))
    s['currentPoint']['x'] = s['currentPoint']['x']  + max1
    
def positionNodes(gr, unvisited, visited, visitors, level):
    if len(unvisited) == 0:
        return;
    row = 0
    visitedNow = []
    only_outputs = True
    for unvis in unvisited:
        if int(unvis) < 4000:
            only_outputs = False
    for n in unvisited:
        if (inputNodesAreVisited(gr['n'][n], visited) or inputNodesAreDefined(gr['n'][n], gr['r'], level)) or (level >0 or int(n) > 2000):
            for visitor in visitors:
                visitor({"node": gr['n'][n],  "level": level, "row": row, "context": funcs})
            row = row + 1
            if only_outputs or int(n) < 4000:
                visitedNow.append(n)
    unvisited = difference(unvisited, visitedNow)
    visited = visited + visitedNow
    return positionNodes(gr, unvisited, visited, visitors, level+1)

def make_runtime(context, rich_graph):
    rows = {}
    rich_graph2 = copy.deepcopy(rich_graph)
    for x in rich_graph2['n']:
        if not rich_graph2['n'][x]['position']['y'] in rows:
            rows[rich_graph2['n'][x]['position']['y']] = []
        rows[rich_graph2['n'][x]['position']['y']].append(x)
    return {
    "rich_graph": rich_graph2,
    "runnable_graph": [value for (key, value) in sorted(rows.items())],
    "context": context,
    "runtime": { "0": rich_graph2['r']}
  }

def run_graph( runtime_graph, ins):
    outs = []
    ndx = 0
    arr_ins = list(ins.values())
    runtime = runtime_graph['runtime']
    runnable = runtime_graph['runnable_graph']
    rich = runtime_graph['rich_graph']
    
    for key in runnable[0]:
        runtime[key] = [arr_ins[ndx]]
        ndx= ndx+1
    ndx = 0
    for x in runnable:
        for y in runnable[ndx]:
            node  = rich['n'][y]
            contxt = runtime_graph['context'][node['id']]
            #print(contxt['meta'])
            #source = contxt['data']['sources']['python']
            args=[]
            if  (1 in node['in']):
                port  =  0
                '''goes in order though the keys: '''
                while ( port+1 in node['in']):
                    x1 = node['in'][port+1]
                    #print(key1, x1, runtime)
                    argt = runtime[str(x1[0])][x1[1]-1]
                    type  = contxt['data']['gapi']['inputs'][port]['type']
                    if (type == "function"):
                        #print(contxt['pfunction']['signature'],port+1,argt)
                        argt  = eval("lambda " + argt) 
                    port  = port +1
                    args.append(argt)
            else:
                args  = runtime[y]
            if (contxt['data']['gapi']['type'] == "function"):
                if (False): # (no graph yet..) len(contxt['data']['graph']) > 0):
                    ans = run_graph (make_runtime (indexed_func) (enrich_graph (indexed_func) (contxt['data']['graph']))) (args);
                    ans = ans.values()
                else:
                    #func = eval("lambda " + source);
                    #print(args)
                    #ans = func(*args);
                    print(node["id"], contxt['data']['gapi']['name'])
                    #instances[node["id"]] = Instance([node["id"]])
                    if (node["id"].find(s['id']) < 0):
                        print("args: ",args)
                        ans = getattr(instances[node["id"]].exports, contxt['data']['gapi']['name'])(*args)
                    else:
                        ans = args[0]
                    print(ans)
            else:
                pass
            runtime[y] = [ans]
            if int(y) >=  4000:
                outs.append(ans)
        ndx = ndx + 1
    #print (runtime)
    return outs;

def fun_graph( context, graph, ins):
    return run_graph (make_runtime (context, enrich_graph (graph)),ins)


def get_graph(id):
    req = requests.get(s['baseURL']+'pfunction/'+id)
    func = req.json()
    gr = func['data']['graph']
    return gr
#enrich_graph(graph)
#print(funcs, graph)

#print(make_runtime (funcs, enrich_graph (graph)))


In [3]:
wasm_bytes = open('simple.wasm', 'rb').read()
instance = Instance(wasm_bytes)
result = instance.exports.sum(15, 7)

print(result)

22


In [4]:
import requests
wasm_bytes = requests.get('http://192.168.1.140:3000/file/2bdde8e4-5abf-4884-8bbf-5779982321e4/source')
#print(wasm_bytes)
instance = Instance(wasm_bytes.content)
result = instance.exports.sum_i8(15, 7)

print(result)

ConnectionError: HTTPConnectionPool(host='192.168.1.140', port=3000): Max retries exceeded with url: /file/2bdde8e4-5abf-4884-8bbf-5779982321e4/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10e70b750>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
%%javascript

var providerNames = {
    '1': "Ethereum Main Network",
'2': "Morden Test network",
'3': "Ropsten Test Network",
'4': "Rinkeby Test Network",
'5': "Goerli Test Network",
'42': "Kovan Test Network"
}

if (typeof window.ethereum !== 'undefined') {

  // Ethereum user detected. You can now use the provider.
  const provider = window['ethereum']
}
console.log(web3.currentProvider.selectedAddress)

//console.log(web3.currentProvider, web3.currentProvider.networkVersion, web3.currentProvider.selectedAddress, web3.currentProvider.enable())//, web3.selectedAddress)

In [ ]:
# with trans

graph12 = {"n":{"101":{"i":101,"id":"9fee587c-f0fa-4478-8487-c73a8e60c597"},"102":{"i":102,"id":"840725c7-7ed0-4315-b7cf-6f0380851bfb"},"103":{"i":103,"id":"43da39a8-5ec8-4a3a-a25a-380515881545"},"3000":{"i":3000,"id":"5bc59e192817116e84bdd8313000"},"3001":{"i":3001,"id":"5bc59e192817116e84bdd8313001"}},"e":[[101,1,102,2],[102,1,103,4],[3000,1,101,1],[3000,1,102,1],[3000,1,101,1],[3000,1,102,1],[3000,1,103,3],[3001,1,102,3],[3001,1,103,5],[101,1,103,1]],"r":[]}
funcs = {}
getFunc(graph12['n'])
egraph2 = make_runtime (funcs, enrich_graph (graph12))
egraph2

In [ ]:
graph1 = {"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88"},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86"},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb"},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"},"106":{"i":106,"id":"5dcaf0cd5427e54a5fe18e8a"},"107":{"i":107,"id":"5dbbf356f18ff7488e9b1096"},"108":{"i":108,"id":"5dbbf356f18ff7488e9b1096"},"109":{"i":109,"id":"5dcadd175427e54a5fe18e88"},"110":{"i":110,"id":"5dcac5c45427e54a5fe18e86"}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3],[102,1,106,1],[104,1,106,2],[103,1,106,3],[105,1,107,2],[106,1,108,2],[107,1,109,1],[108,1,109,2],[109,1,110,1]],"r":[]}
#graph1 = {"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88"},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86"},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb"},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"},"106":{"i":106,"id":"5dcaf0cd5427e54a5fe18e8a"}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3],[102,1,106,1],[104,1,106,2],[103,1,106,3]],"r":[]}
#graph1 = get_graph("5dcaa55a3f65440d349525f2")
#graph1 = get_graph("5dcb277d3f65440d349525f3")
#graph1 = {"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88"},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86"},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb"},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3]],"r":[]}
print(graph1)

funcs = {}
getFunc(graph1['n'])

In [ ]:
funcs = {}
getFunc(graph1['n'])

In [ ]:
import ipywidgets as widgets
#from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import *
from IPython.display import display
from IPython.display import YouTubeVideo


egraph = make_runtime (funcs, enrich_graph (graph1))
#print(egraph)


    
def operate(gr):
    inputs = []
    outputs = []
    consts = []
    def execute_gr(v):
        ins = {}
        ind = 0
        for item in inputs:
            #print(item.value)
            try:
                i = json.loads("["+item.value+"]")[0]
            except:
                i = json.loads("[\""+item.value+"\"]")[0]
            ins[str(ind)] = i
            ind = ind +1
        print(ins)
        #outs = fun_graph(funcs, gr, ins)
        try:
            outs = run_graph(gr, ins)
        except:
            outs =[]
        ind  = 0
        for item in outputs:
            #print(item.value)
            #ins[ind] = json.loads(item.value)
            try:
                item.value = str(outs[ind])
            except:
                pass
            ind = ind +1
    for key in gr['runnable_graph']:
        for key2 in key:
            #print(key2)
            port = funcs[gr['rich_graph']['n'][key2]['id']]['data']['gapi']['outputs'][0]
            if int(key2) > 2999 and int(key2) <4000:
                #node
                #print(key2)
                tinput = widgets.Text(
                    value='',
                    placeholder='Input: '+port['type'],
                    description= port['name']+': '+port['type'],
                    disabled=False
                )
                tinput.observe(execute_gr, names='value')
                inputs.append(tinput)
            if int(key2) > 3999:
                outputs.append(widgets.Text(
                    value='',
                    placeholder='Output: '+port['type'],
                    description= port['name']+': '+port['type'],
                    disabled=True
                ))
    for key in gr['runtime']:
        #print(gr['runtime'][key])
        consts.append(widgets.Text(
                    value = str(gr['runtime'][key]),
                    description='Constant:',
                    disabled=True
                )
        )
    display(widgets.HBox([widgets.VBox(inputs + consts), widgets.VBox(outputs)]))
    
operate(egraph)

In [ ]:
run_graph(egraph, {'0': 10, '1': 2, '2': 3})

In [ ]:
getattr(instances["39879128-983d-43fe-ab42-055aaa340f2d"].exports, "sum_i8")(*[3,7])

In [ ]:
display(YouTubeVideo('ME0GZWDfX94'))
    


In [ ]:
"fffert".find("f5f")

In [ ]:
type(YouTubeVideo('eWzY2nGfkXk'))

In [ ]:
%%HTML
<script src="https://cdn.jsdelivr.net/npm/d3@5.15.0/dist/d3.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/sanctuary@3.0.0/index.js"></script>
<script src="https://cdn.jsdelivr.net/npm/sanctuary-def@0.21.1/index.js"></script>




In [ ]:
gr_txt = json.dumps(egraph)

j = json.loads(gr_txt)
grains = widgets.Textarea(
    value= gr_txt,
    placeholder='Graph JSON',
    description='Graph JSON:',
    disabled=False
)
display(grains)

In [ ]:
from IPython.core.display import HTML
HTML('''
<link href="https://fonts.googleapis.com/css?family=Roboto+Condensed&display=swap" rel="stylesheet">
<div id="graph1"><canvas id="graph"></canvas></div>
''')

In [ ]:
%%javascript
//#https://cdn.jsdelivr.net/npm/sanctuary@3.0.0/index.js <script src="https://raw.githubusercontent.com/sanctuary-js/sanctuary/master/dist/bundle.js"></script>
//console.log()

function graph_edit(ctx, r_graph){ 
  const height = Math.ceil(width * screen.height / screen.width);
  //const ctx = DOM.context2d(width, height)
  // const ctx  = canvas.node().getContext('2d');
  //d3.select(ctx.canvas).call(d3.drag().on("start", dstart).on("drag", ddrag).on("end", dend));
  
  let dnodes = {}
  let rows = r_graph.runnable_graph.length
  let cols = 0
  //mutable gr = r_graph
  
  for (const nodes of r_graph.runnable_graph) {
    for (const node of nodes) {
      let node_i = r_graph.rich_graph.n[node]
      let node_x = r_graph.context[node_i.id]
      let len = Math.max(node_x.data.gapi.inputs.length, node_x.data.gapi.outputs.length)
      dnodes[node] = {x: node_i.position.x, y: node_i.position.y, container: node_x.pclassid, name: node_x.data.gapi.name, len: len, context: node_x.data.gapi, i:node}
      if (cols < node_i.position.x  + len) {
        cols = node_i.position.x  + len
      }
    }
  }
  
  current_stage.settings = {width:width, height: height, nodes: dnodes, edges: r_graph.rich_graph.e, matrix: [rows,cols], transform: {x:0, y:0, l:1, f:0.00}, r_graph: r_graph}
  
  //while (current_edge.pos.length > 0) {
    //graph_show(ctx, width,  height, dnodes,  graph1.rich_graph.e, [rows,cols]);
    //yield ctx.canvas
  //}
  return graph_show(ctx, width,  height, dnodes,  r_graph.rich_graph.e, [rows,cols], current_stage.settings.transform, current_stage.settings.r_graph); // x:0, y:0, l:1, f:0.00
  
  
  //ctx.canvas;
}

function graph_show(ctx2, width,  height, nodes,  edges, matrix, transform, r_graph){
  //mutable 
  ctx2.clearRect(0, 0, width, height);
  ctx2.fillStyle="#fff"
  rect1([ctx2,width/2,height/2,width,height])
  current_stage.settings = {ctx: ctx2, width:width, height: height, nodes: nodes, edges:edges, matrix:matrix, transform:transform, r_graph: r_graph}
  //console.log(JSON.stringify( current_stage))
  //if (0) {
    //drawExactArrow(ctx, current_edge.pos[0], current_edge.pos[1], current_edge.pos[2],
     //current_edge.pos[3], "number", 1 )
  //}
  console.log( current_stage.settings.r_graph.rich_graph.init)
  let ports_pos = {}
  for (const node in nodes) {
    let n = nodes[node]
    //console.log(node, n)
    node_show(ctx2, n, matrix, [width,height], ports_pos)
  }
  
  //console.log(ports_pos)
  
  for (const edge of edges) {
    
      let e = edge
      let p1 = ports_pos[[e[0], e[1], 1]]
      let p2 = ports_pos[[e[2], e[3], 0]]
      let mid = [(p1.x+p2.x)/2,  (p1.y+p2.y)/2]
      let r = 4
      //console.log(edge, p1,p2)
      drawExactArrow(ctx2, p1.x, p1.y, p2.x, p2.y, p1.type, 1)
      targets.for_click_edge[e]= {t: mid[1]-r,  b: mid[1]+r , l:  mid[0]-r, r: mid[0]+r}
    }
  
  //return ctx2
  
}

function node_show(ctx, node, matrix, dims, ports_pos){
  
  let scale=1
  let x = node.x
  let y = node.y
  let pos = pos_calculate(x, y, node.len, matrix, dims)
  let nw = pos.l
  let nh = pos.h
  
  
  ports_show(ctx, pos, node.context, ports_pos, node.i)
  ctx.fillStyle="#ccc"
  rect1([ctx,pos.x,pos.y,nw,pos.h,pos.h/2])
  targets.for_click[[node.i, 1]] = {l: pos.x-nw/2, r:  pos.x+nw/2-16 , t: pos.y-pos.h/2 , b: pos.y+pos.h/2}
  ctx.fillStyle="#fcc"
  rect1([ctx,pos.x+nw/2-8,pos.y,16,16,8])
  targets.for_click[[node.i, 0]] = {l: pos.x+nw/2-16, r:  pos.x+nw/2 , t: pos.y-pos.h/2 , b: pos.y+pos.h/2}
  ctx.beginPath()
  ctx.textAlign = "center";
  //ctx.font = (18*scale-9)+"px Roboto Condensed"
  //context.lineJoin = "round";
  ctx.fillStyle = "#000000"
  //ctx.fillText(shorten(node.container,max*5+2), pos.x, pos.y-2 );
  ctx.font = (12*scale+4)+"px Roboto Condensed"
  ctx.fillText(shorten(node.container,node.len*2+2)+": "+shorten(node.name,node.len*10+4), pos.x, pos.y+6 );
  ctx.fill()
}

function ports_show(ctx, pos, node_x, ports_pos,i){
  let ndx=0, nw=20
  for (let port1 of node_x.inputs){
    //console.log(port1, pos.x+pos.l+50*ndx, pos.y-20)
    ctx.fillStyle = types[port1.type].color
    let x = pos.x-pos.l/2+pos.steps.w*(ndx+0.5)
    let y = pos.y-pos.h/2
    rect1([ctx,x, y,nw,nw,10,10,0,0])
    ctx.save();
    ctx.translate( x, y);
    ctx.rotate( -Math.PI / 4 );
    ctx.font = (8/1+4)+"px Roboto Condensed";
    ctx.fillStyle = "#aaaaaa";; // red
    ctx.textAlign = "left";
    ctx.fillText( port1.name, 0,0 );
    ctx.restore();
    ports_pos[[i,ndx+1,0]]={x: x,y: y, type: port1.type}
    targets.end_drag[[i,ndx+1]] = {x: x,  y: y, port: {i:i, port:ndx+1, type:port1.type}}
    ndx++
  }
  ndx=0
  for (let port1 of node_x.outputs){
    //console.log(port1)
    ctx.fillStyle = types[port1.type].color
    let x = pos.x-pos.l/2+pos.steps.w*(ndx+0.5)
    let y = pos.y+pos.h/2
    rect1([ctx, x, y,nw,nw,0,0,10,10])
    ports_pos[[i,ndx+1,1]]={x: x, y: y, type: port1.type}
    targets.for_drag[[i,ndx+1]] = {l: x-10,  r: x+10 ,t: y,b: y+10, port: {i:i, port:ndx+1, type:port1.type}}
    ndx++
  }
}

function shorten(txt, noChar){
  let len = txt.length
  if ( len < noChar) return txt;
  let half = Math.floor(noChar/2)
  //// console.log(len,half)
  let ans = txt.slice(0, half)+ ".." +txt.slice(-half)
  return ans

}

function drawExactArrow(ctx2, x1, y1, x2, y2, type, scale){
    var midx = (x2+x1)/2
    var midy = (y2+y1)/2
    let context = ctx2
    //let color  = types[type].color
    let srt  = "M"+x1+","+y1+"C"+x1+"," +midy+"," +
    midx+","+ midy+","+ midx+","+ midy+
    "C"+midx+"," +midy+","+ x2+","+ midy+"," +x2+","+ y2
    //let scale = 1
    //context.moveTo(x1, y1)
    context.lineCap = "round"
    context.strokeStyle  =  types[type].color? types[type].color : "#aaaaaa"
    context.lineWidth = 1*scale;
    context.stroke(new Path2D(srt));
    context.closePath();
    //context.moveTo(midx, midy)
    context.beginPath()
    context.arc(midx, midy, 3, 0, Math.PI*2, false);
    context.fillStyle = '#DDAAAA';
    context.fill();
    context.closePath();
}

function rect1(args) {
  const ctx = args[0];
  const x = args[1];
  const y = args[2];
  const w = args[3];
  const h = args[4];
  let tl = args[5];
  let tr = args[6];
  let br = args[7];
  let bl = args[8];
  
  //console.log(ctx)
  
  ctx.beginPath();

  if (typeof tl === 'undefined') {
    // No rounded corners
    ctx.rect(x-w/2, y-h/2, w, h);
  } else {
    // At least one rounded corner
    // Set defaults when not specified
    if (typeof tr === 'undefined') {
      tr = tl;
    }
    if (typeof br === 'undefined') {
      br = tr;
    }
    if (typeof bl === 'undefined') {
      bl = br;
    }

    // corner rounding must always be positive
    const absW = Math.abs(w);
    const absH = Math.abs(h);
    const hw = absW / 2;
    const hh = absH / 2;

    // Clip radii
    if (absW < 2 * tl) {
      tl = hw;
    }
    if (absH < 2 * tl) {
      tl = hh;
    }
    if (absW < 2 * tr) {
      tr = hw;
    }
    if (absH < 2 * tr) {
      tr = hh;
    }
    if (absW < 2 * br) {
      br = hw;
    }
    if (absH < 2 * br) {
      br = hh;
    }
    if (absW < 2 * bl) {
      bl = hw;
    }
    if (absH < 2 * bl) {
      bl = hh;
    }

    // Draw shape
    ctx.beginPath();
    ctx.moveTo(x + tl-w/2, y- h/2);
    ctx.arcTo(x + w/2, y-h/2, x + w/2, y + h/2, tr);
    ctx.arcTo(x + w/2, y + h/2, x-w/2, y + h/2, br);
    ctx.arcTo(x-w/2, y + h/2, x-w/2, y-h/2, bl);
    ctx.arcTo(x-w/2, y-h/2, x + w/2, y-h/2, tl);
    ctx.closePath();
  }
    ctx.fill();

  return ctx;
};

function pos_calculate(x, y, l, matrix, dims){
  let marginw = width/30
  let marginh = 0
  let minh = 16
  let stepw = (dims[0]-2*marginw)/(matrix[1]+0)
  let steph = (dims[1]-2*marginh+minh)/(matrix[0]-1)//-minh/2
  return {x: x*stepw+marginw+l*stepw/2, y: y*steph-minh/2, l: l*stepw, h: minh, steps: {w:stepw, h:steph}}
}

function doit(){
 
  const height = Math.ceil(width * screen.height / screen.width);
  //let canvas1 = d3.select('canvas').attr('width', width).attr('height', height);
  let canvas1 = d3.select('#graph').attr('width', width).attr('height', height);
  //const ctx = DOM.context2d(width, height);
  const ctx1  =canvas1.node().getContext('2d');
  //mutable graph1 = runnable(g0);
  //d3.select(ctx1.canvas).call(d3.drag().on("start", dstart).on("drag", ddrag).on("end", dend));
  //return );
  current_stage.settings.ctx = ctx1;
  graph_edit(current_stage.settings.ctx, current_stage.settings.r_graph)
}

var ta = document.getElementsByClassName("widget-textarea")[0];
var grrin = ta.getElementsByTagName("TEXTAREA")[0].value
            // document.getElementById("bbd6bfe3-73cf-4411-95f6-e61a12e7da7e").value
console.log(ta)
var r_graph = JSON.parse(grrin)
console.log(grrin)
var current_stage = {"settings":{"transform":{},
    "r_graph": r_graph //{"rich_graph":{"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88","out":{"1":[[102,1]]},"in":{"1":[3000,1],"2":[3001,1]},"position":{"x":0,"y":1},"svg_id":"","edges":[]},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86","out":{"1":[[103,1],[105,1],[106,1]]},"in":{"1":[101,1]},"position":{"x":0,"y":2},"svg_id":"","edges":[]},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb","out":{"1":[[104,1],[106,3]]},"in":{"1":[102,1]},"position":{"x":0,"y":3},"svg_id":"","edges":[]},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc","out":{"1":[[105,3],[106,2]]},"in":{"1":[103,1]},"position":{"x":0,"y":4},"svg_id":"","edges":[]},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a","out":{"1":[[107,2]]},"in":{"1":[102,1],"2":[3002,1],"3":[104,1]},"position":{"x":0,"y":5},"svg_id":"","edges":[]},"106":{"i":106,"id":"5dcaf0cd5427e54a5fe18e8a","out":{"1":[[108,2]]},"in":{"1":[102,1],"2":[104,1],"3":[103,1]},"position":{"x":3,"y":5},"svg_id":"","edges":[]},"107":{"i":107,"id":"5dbbf356f18ff7488e9b1096","out":{"1":[[109,1]]},"in":{"1":[3003,1],"2":[105,1]},"position":{"x":0,"y":6},"svg_id":"","edges":[]},"108":{"i":108,"id":"5dbbf356f18ff7488e9b1096","out":{"1":[[109,2]]},"in":{"1":[3004,1],"2":[106,1]},"position":{"x":2,"y":6},"svg_id":"","edges":[]},"109":{"i":109,"id":"5dcadd175427e54a5fe18e88","out":{"1":[[110,1]]},"in":{"1":[107,1],"2":[108,1]},"position":{"x":0,"y":7},"svg_id":"","edges":[]},"110":{"i":110,"id":"5dcac5c45427e54a5fe18e86","out":{"1":[[4000,1]]},"in":{"1":[109,1]},"position":{"x":0,"y":8},"svg_id":"","edges":[]},"3000":{"edges":[],"i":3000,"id":"5bc59e192817116e84bdd8313000","in":{},"out":{"1":[[101,1]]},"position":{"x":0,"y":0},"svg_id":""},"3001":{"edges":[],"i":3001,"id":"5bc59e192817116e84bdd8313001","in":{},"out":{"1":[[101,2]]},"position":{"x":1,"y":0},"svg_id":""},"3002":{"edges":[],"i":3002,"id":"5bc59e192817116e84bdd8313002","in":{},"out":{"1":[[105,2]]},"position":{"x":2,"y":0},"svg_id":""},"3003":{"edges":[],"i":3003,"id":"5bc59e192817116e84bdd8313003","in":{},"out":{"1":[[107,1]]},"position":{"x":3,"y":0},"svg_id":""},"3004":{"edges":[],"i":3004,"id":"5bc59e192817116e84bdd8313004","in":{},"out":{"1":[[108,1]]},"position":{"x":4,"y":0},"svg_id":""},"4000":{"i":4000,"svg_id":"","id":"5bc59e192817116e84bdd8314000","in":{"1":[110,1]},"out":{},"position":{"x":0,"y":9},"edges":[]}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3],[102,1,106,1],[104,1,106,2],[103,1,106,3],[105,1,107,2],[106,1,108,2],[107,1,109,1],[108,1,109,2],[109,1,110,1],[3000,1,101,1],[3001,1,101,2],[3002,1,105,2],[3003,1,107,1],[3004,1,108,1],[110,1,4000,1]],"r":[],"init":{"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88"},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86"},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb"},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"},"106":{"i":106,"id":"5dcaf0cd5427e54a5fe18e8a"},"107":{"i":107,"id":"5dbbf356f18ff7488e9b1096"},"108":{"i":108,"id":"5dbbf356f18ff7488e9b1096"},"109":{"i":109,"id":"5dcadd175427e54a5fe18e88"},"110":{"i":110,"id":"5dcac5c45427e54a5fe18e86"}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3],[102,1,106,1],[104,1,106,2],[103,1,106,3],[105,1,107,2],[106,1,108,2],[107,1,109,1],[108,1,109,2],[109,1,110,1]],"r":[]}},"runnable_graph":[[3000,3001,3002,3003,3004],[101],[102],[103],[104],[105,106],[107,108],[109],[110],[4000]],"context":{"5ddd437ffefb0c4ed7ef3472":{"_id":"5ddd437ffefb0c4ed7ef3472","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"executionEnvVersion()","gapi":{"constant":true,"inputs":[],"name":"executionEnvVersion","outputs":[{"type":"string","name":"eeVersion"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript.rust":"() => rust.executionEnvVersion()","javascript":"() => typeof process !== 'undefined' && process.version ? \`node.js version: \${process.version}\` : 'javascript: unknown'"}},"categories":{"languages":["javascript.rust","javascript"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dbbf356f18ff7488e9b1096":{"_id":"5dbbf356f18ff7488e9b1096","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"map(function,number[])","gapi":{"constant":true,"inputs":[{"type":"function","name":"function"},{"type":"number[]","name":"array"}],"name":"map","outputs":[{"type":"number[]","name":"array"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"S.map","python":"a,b : list(map(a,b))"}},"categories":{"languages":["javascript","python"],"categories":[]},"timestamp":"2019-10-31T09:14:08.756Z"},"5dbd6dee052a148d62eae282":{"_id":"5dbd6dee052a148d62eae282","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"add(number,number)","gapi":{"constant":true,"inputs":[{"type":"number","name":"term1"},{"type":"number","name":"term2"}],"name":"add","outputs":[{"type":"number","name":"add"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"S.add","python":"a,b : a + b"}},"categories":{"languages":["javascript","python"],"categories":[]},"timestamp":"2019-11-02T08:07:17.930Z"},"5dc1ac9333634f14a7cca672":{"_id":"5dc1ac9333634f14a7cca672","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"filter(function,number[])","gapi":{"constant":true,"inputs":[{"type":"function","name":"function"},{"type":"number[]","name":"array"}],"name":"filter","outputs":[{"type":"number[]","name":"array"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"S.filter","python":"a,b : list(filter(a,b))"}},"categories":{"languages":["javascript","python"],"categories":[]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dc1ac9333634f14a7cca673":{"_id":"5dc1ac9333634f14a7cca673","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"min(number,number)","gapi":{"constant":true,"inputs":[{"type":"number","name":"a"},{"type":"number","name":"b"}],"name":"min","outputs":[{"type":"number","name":"min"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a,b) => Math.min(a,b)","python":"a,b : min(a,b)"}},"categories":{"languages":["javascript","python"],"categories":[]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dc1ac9333634f14a7cca674":{"_id":"5dc1ac9333634f14a7cca674","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"max(number,number)","gapi":{"constant":true,"inputs":[{"type":"number","name":"a"},{"type":"number","name":"b"}],"name":"max","outputs":[{"type":"number","name":"max"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a,b) => Math.max(a,b)","python":"a,b : max(a,b)"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dc32dbfa047684155dff36d":{"_id":"5dc32dbfa047684155dff36d","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"pow(number,number)","gapi":{"constant":true,"inputs":[{"type":"number","name":"power"},{"type":"number","name":"number"}],"name":"pow","outputs":[{"type":"number","name":"power"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"S.pow","python":"a,b : a * b"}},"categories":{"languages":["javascript","python"],"categories":[]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dca7a5c5427e54a5fe18e83":{"_id":"5dca7a5c5427e54a5fe18e83","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"min(number[])","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array"}],"name":"min","outputs":[{"type":"number","name":"min"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"a => Math.min(...a)","python":"a,b : min(a)"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dca7aa05427e54a5fe18e84":{"_id":"5dca7aa05427e54a5fe18e84","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"max(number[])","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array"}],"name":"max","outputs":[{"type":"number","name":"max"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"a => Math.max(...a)","python":"a : max(a)"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dca8b125427e54a5fe18e85":{"_id":"5dca8b125427e54a5fe18e85","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"find(function,number[])","gapi":{"constant":true,"inputs":[{"type":"function","name":"function"},{"type":"number[]","name":"array"}],"name":"find","outputs":[{"type":"number","name":"found"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(f, a) => a.find(f)","python":"f,a: list(filter(f,a))[0]"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcaa55a3f65440d349525f2":{"_id":"5dcaa55a3f65440d349525f2","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"mapmax(number[])","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"number[]"}],"name":"mapmax","outputs":[{"type":"number","name":"mapmax"}],"payable":false,"stateMutability":"pure","type":"function"},"sources":{"javascript":"","python":""},"graph":{"n":{"1":{"i":1,"id":"5dbbf356f18ff7488e9b1096"},"101":{"i":101,"id":"5dca7aa05427e54a5fe18e84"}},"e":[[0,1,1,1],[1,1,101,1]],"r":["h : h*9"]}},"categories":{"languages":["javascript"]},"timestamp":"2019-11-11T22:57:03.346Z"},"5dcac5c45427e54a5fe18e86":{"_id":"5dcac5c45427e54a5fe18e86","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"sort(number[])","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array"}],"name":"sort","outputs":[{"type":"number[]","name":"sorted"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a) => a.sort((a,b)=>a-b)","python":"a: sorted(a)"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcadd175427e54a5fe18e88":{"_id":"5dcadd175427e54a5fe18e88","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"concat(number[],number[])","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array1"},{"type":"number[]","name":"array2"}],"name":"concat","outputs":[{"type":"number[]","name":"concatenated"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a,b) => a.concat(b)","python":"a,b: list(a)+list(b)"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcadec35427e54a5fe18e89":{"_id":"5dcadec35427e54a5fe18e89","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"join(string,string[])","gapi":{"constant":true,"inputs":[{"type":"string","name":"separator"},{"type":"string[]","name":"strArray"}],"name":"join","outputs":[{"type":"string","name":"joined"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a,b) => b.join(a)","python":"a,b: a.join(b)"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcaf0cd5427e54a5fe18e8a":{"_id":"5dcaf0cd5427e54a5fe18e8a","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"slice(number[],number,number)","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array"},{"type":"number","name":"start"},{"type":"number","name":"stop"}],"name":"slice","outputs":[{"type":"number[]","name":"sliced"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(arr,start,end) => arr.slice(start,end)","python":"arr,start,end: arr[start:end]"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcafee95427e54a5fe18e8b":{"_id":"5dcafee95427e54a5fe18e8b","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"splice(number[],number,number,number[])","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array"},{"type":"number","name":"start"},{"type":"number","name":"deleteCount"},{"type":"number[]","name":"added"}],"name":"splice","outputs":[{"type":"number[]","name":"sliced"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(arr,start,deleteCount,added=[]) => {arr.splice(start,deleteCount,...added);return arr;}","python":"arr,start,deleteCount,added=[]: arr[0:start]+added+arr[start+deleteCount:]"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcb13895427e54a5fe18e8c":{"_id":"5dcb13895427e54a5fe18e8c","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"subtract(number,number)","gapi":{"constant":true,"inputs":[{"type":"number","name":"minuend"},{"type":"number","name":"subtrahend"}],"name":"subtract","outputs":[{"type":"number","name":"result"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a,b) => a - b","python":"a,b: a - b"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcb146d5427e54a5fe18e8d":{"_id":"5dcb146d5427e54a5fe18e8d","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"neg(number)","gapi":{"constant":true,"inputs":[{"type":"number","name":"number"}],"name":"neg","outputs":[{"type":"number","name":"result"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a) => -a","python":"a: -a"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcb277d3f65440d349525f3":{"_id":"5dcb277d3f65440d349525f3","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"testgraph1(number[],number[],number,number)","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array1"},{"type":"number[]","name":"array2"},{"type":"number","name":"start"},{"type":"number","name":"end"}],"name":"testgraph1","outputs":[{"type":"number[]","name":"output"}],"payable":false,"stateMutability":"pure","type":"function"},"sources":{"javascript":"","python":""},"graph":{"n":{"102":{"i":102,"id":"5dcadd175427e54a5fe18e88"},"103":{"i":103,"id":"5dcac5c45427e54a5fe18e86"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"}},"e":[[102,1,103,1],[103,1,105,1]],"r":[""]}},"categories":{"languages":["javascript"]},"timestamp":"2019-11-11T22:57:03.346Z"},"5dcfebb36a2b0f10e93b09cb":{"_id":"5dcfebb36a2b0f10e93b09cb","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"length(number[])","gapi":{"constant":true,"inputs":[{"type":"number[]","name":"array"}],"name":"length","outputs":[{"type":"number","name":"length"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a) => a.length","python":"a: len(a)"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dcfec8b6a2b0f10e93b09cc":{"_id":"5dcfec8b6a2b0f10e93b09cc","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"half(number)","gapi":{"constant":true,"inputs":[{"type":"number","name":"number"}],"name":"half","outputs":[{"type":"number","name":"half"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(a) => a / 2","python":"a: a // 2"}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5dd852136b366ea53c42e177":{"_id":"5dd852136b366ea53c42e177","pclassid":"5dbaa731f18ff7488e9b108b","pfunction":{"signature":"storage_allDocs(string)","gapi":{"constant":true,"inputs":[{"type":"string","name":"database"}],"name":"storage_allDocs","outputs":[{"type":"object[]","name":"docs"}],"payable":false,"stateMutability":"pure","type":"function"},"graph":{},"sources":{"javascript":"(dbname) => window.plugems.list.storage.allDocs(dbname).then(resp => resp.rows.map(row => row.doc))","python":""}},"categories":{"languages":["javascript","python"]},"timestamp":"2019-11-06T19:22:53.807Z"},"5bc59e192817116e84bdd8313000":{"_id":"5bc59e192817116e84bdd8313000","pclassid":"5c95397d4212cc40afeec914","pfunction":{"signature":"io","sources":{"javascript":"f=>f"},"graph":{},"gapi":{"constant":true,"inputs":[{"name":"i","type":"number[]"}],"name":"io","outputs":[{"name":"array1","type":"number[]"}],"payable":false,"stateMutability":"view","type":"function"},"chainids":["3"]},"categories":{"tags":["Pipeline Demo Package","ethpm","pipeline","ethpm"]},"timestamp":"2019-03-22T14:38:36.112Z","graph":{}},"5bc59e192817116e84bdd8313001":{"_id":"5bc59e192817116e84bdd8313001","pclassid":"5c95397d4212cc40afeec914","pfunction":{"signature":"io","sources":{"javascript":"f=>f"},"graph":{},"gapi":{"constant":true,"inputs":[{"name":"i","type":"number[]"}],"name":"io","outputs":[{"name":"array2","type":"number[]"}],"payable":false,"stateMutability":"view","type":"function"},"chainids":["3"]},"categories":{"tags":["Pipeline Demo Package","ethpm","pipeline","ethpm"]},"timestamp":"2019-03-22T14:38:36.112Z","graph":{}},"5bc59e192817116e84bdd8313002":{"_id":"5bc59e192817116e84bdd8313002","pclassid":"5c95397d4212cc40afeec914","pfunction":{"signature":"io","sources":{"javascript":"f=>f"},"graph":{},"gapi":{"constant":true,"inputs":[{"name":"i","type":"number"}],"name":"io","outputs":[{"name":"start","type":"number"}],"payable":false,"stateMutability":"view","type":"function"},"chainids":["3"]},"categories":{"tags":["Pipeline Demo Package","ethpm","pipeline","ethpm"]},"timestamp":"2019-03-22T14:38:36.112Z","graph":{}},"5bc59e192817116e84bdd8313003":{"_id":"5bc59e192817116e84bdd8313003","pclassid":"5c95397d4212cc40afeec914","pfunction":{"signature":"io","sources":{"javascript":"f=>f"},"graph":{},"gapi":{"constant":true,"inputs":[{"name":"i","type":"function"}],"name":"io","outputs":[{"name":"function","type":"function"}],"payable":false,"stateMutability":"view","type":"function"},"chainids":["3"]},"categories":{"tags":["Pipeline Demo Package","ethpm","pipeline","ethpm"]},"timestamp":"2019-03-22T14:38:36.112Z","graph":{}},"5bc59e192817116e84bdd8313004":{"_id":"5bc59e192817116e84bdd8313004","pclassid":"5c95397d4212cc40afeec914","pfunction":{"signature":"io","sources":{"javascript":"f=>f"},"graph":{},"gapi":{"constant":true,"inputs":[{"name":"i","type":"function"}],"name":"io","outputs":[{"name":"function","type":"function"}],"payable":false,"stateMutability":"view","type":"function"},"chainids":["3"]},"categories":{"tags":["Pipeline Demo Package","ethpm","pipeline","ethpm"]},"timestamp":"2019-03-22T14:38:36.112Z","graph":{}},"5bc59e192817116e84bdd8314000":{"_id":"5bc59e192817116e84bdd8314000","pclassid":"5c95397d4212cc40afeec914","pfunction":{"signature":"io","sources":{"javascript":"f=>f"},"graph":{},"gapi":{"constant":true,"inputs":[{"name":"sorted","type":"number[]"}],"name":"io","outputs":[{"name":"o","type":"number[]"}],"payable":false,"stateMutability":"view","type":"function"},"chainids":["3"]},"categories":{"tags":["Pipeline Demo Package","ethpm","pipeline","ethpm"]},"timestamp":"2019-03-22T14:38:36.112Z","graph":{}}},"runtime":{"0":[]}}
   }}
console.log(current_stage)
var width = 800
var types = {
  "number": {color: "#355"},
    "i8": {color: "#355"},
  "number[]": {color: "#463"},
  "function": {color: "#562"},
  "string": {color: "#662"},
}

var targets = {
  for_click: {},
  for_click_edge: {},
  for_drag: {},
  end_drag:{},
  for_drop: {}
}

doit()

In [ ]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [ ]:
from ctypes import *

#load the shared object file
adder = CDLL('./adder.so')

#Find sum of integers
res_int = adder.add_int(4,5)
print ("Sum of 4 and 5 = " + str(res_int))

#Find sum of floats
a = c_float(5.5)
b = c_float(4.1)

add_float = adder.add_float
add_float.restype = c_float
print ("Sum of 5.5 and 4.1 = ", str(add_float(a, b)))

In [ ]:
#from web3.auto import w3
#WEB3_INFURA_PROJECT_ID="84c234eeec774a61af6adc0a366967b5"
from web3 import Web3, HTTPProvider

w3 = Web3(Web3.HTTPProvider("https://ropsten.infura.io/v3/84c234eeec774a61af6adc0a366967b5"))
#from web3.auto.infura import w3
#w3.eth.blockNumber

c_abi = json.loads('[{"constant":false,"inputs":[{"name":"vendor","type":"address"},{"name":"product_id","type":"uint256"}],"name":"registerVendor","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"product_id","type":"uint256"},{"indexed":false,"name":"vendor","type":"address"}],"name":"VendorRegistered","type":"event"},{"constant":true,"inputs":[{"name":"product_id","type":"uint256"}],"name":"getVendor","outputs":[{"name":"vendor","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"name":"table","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"}]')
c_address = Web3.toChecksumAddress("0x274782b8155ddcc9d868f9e676e6688d78c22488")

store_var_contract = w3.eth.contract(
   address= c_address,
   abi= c_abi)

gas_estimate = store_var_contract.functions['getVendor'](1).call()
print("Output of getVendor: {0}\n".format(gas_estimate))
#export WEB3_INFURA_PROJECT_ID = "84c234eeec774a61af6adc0a366967b5"
# export WEB3_INFURA_PROJECT_ID=84c234eeec774a61af6adc0a366967b5

w3.eth.blockNumber
#w3.eth.accounts[0]

In [ ]:
def show_func(address, abi, func):
    inputs=[]
    addr =  widgets.Text(
            value='0xe5135bC48c3696be2e77F51BD85D1EC91DE7ADe7',
            placeholder='Addr: ',
            description= "Address",
            disabled=False
        )
    outputs=[]
    outs=[]
    tools=[]
    funcs = []
    ndx = 0
    j = 0
    ans = 0
    contract = w3.eth.contract(
       address= address,
       abi= abi)
    def sel_func(f):
        print(f)
        w.close()
        show_func(address, abi, f["new"])
    def run_func(v):
        ins = {}
        ind = 0
        outs = []
        for item in inputs:
            #print(item.value)
            try:
                i = json.loads("["+item.value+"]")[0]
            except:
                i = json.loads("[\""+item.value+"\"]")[0]
            ins[str(ind)] = i
            ind = ind +1
        #print(ins)
        #outs = fun_graph(funcs, gr, ins)
        try:
            #outs = run_graph(gr, ins)
            #print(restr(ins))
            ans = contract.functions[func](*list(ins.values())).call({send:addr.value})
            #print(ans)
            #outs[0]=ans
            #if type(ans) is list: outs = ans else: outs[0]=ans
        except:
            ans =[]
        if type(ans) is list: 
            outs = ans 
        else: 
            outs.append(ans)
        #print(outs)
        ind  = 0
        for item in outputs:
            #print(item.value)
            #ins[ind] = json.loads(item.value)
            try:
                item.value = str(outs[ind])
            except:
                pass
            ind = ind +1
    for i in abi:
        if "name" in i:
            ndx = j
            name = i["name"]
            if func == i["name"]:
                ndx = j
        else:
            name = "constructor"
        j = j + 1
        funcs.append(name)
    funcsel = widgets.Dropdown(
        options= funcs,
        value= func,
        description='Function:',
    )
    funcsel.observe(sel_func, names='value')
    for i in abi[ndx]["inputs"]:
        tinput = widgets.Text(
            value='',
            placeholder='Input: '+i['type'],
            description= i['name']+': '+i['type'],
            disabled=False
        )
        tinput.observe(run_func, names='value')
        inputs.append(tinput)
    for i in abi[ndx]["outputs"]:
        toutput = widgets.Text(
            value='',
            placeholder='Output: '+i['type'],
            description= i['name']+': '+i['type'],
            disabled=False
        )
        #tinput.observe(execute_gr, names='value')
        outputs.append(toutput)
    tools.append(widgets.Button(
        description='Call',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    ))
    w = widgets.HBox([widgets.VBox(inputs + tools), widgets.VBox([funcsel]+[addr] + outputs)])
    display(w)
    #w.close()
   


show_func(c_address, c_abi, "getVendor")

In [ ]:
graph1 = {"n":{"101":{"i":101,"id":"39879128-983d-43fe-ab42-055aaa340f2d"},"102":{"i":102,"id":"4a19014e-16ef-4e51-9c69-0d0c189b44b7"}},"e":[[101,1,102,1]],"r":[]}
graph = {"n":{"101":{"i":101,"id":"39879128-983d-43fe-ab42-055aaa340f2d"},"102":{"i":102,"id":"4a19014e-16ef-4e51-9c69-0d0c189b44b7"},"3000":{"i":3000,"id":"5bc59e192817116e84bdd8313000"}},"e":[[101,1,102,1],[3000,1,101,1],[3000,1,101,2]],"r":[]}
context_txt = """{
  "39879128-983d-43fe-ab42-055aaa340f2d": {
    "_id": "39879128-983d-43fe-ab42-055aaa340f2d",
    "pclassid":"5c95397d4212cc40afeec914",
    "pfunction":{
      "signature":"sum(uint8)",
      "sources": {"python":"lambda a,b: a+b"},
      "graph":{},
      "gapi":{"constant": true,"inputs":[{"type":"uint8", "name":"a"},{"type":"uint8", "name":"b"}],"name":"io","outputs":[{"type":"uint8", "name":"sum"}],"payable": false,"stateMutability":"view","type":"function"}
    },
    "timestamp":"2019-03-22T14:38:36.112Z",
    "pclass": {
      "_id": "5c95397d4212cc40afeec914",
      "name": "sums",
      "type": "python",
      "deployment": "unknown"
    }
  },
  "4a19014e-16ef-4e51-9c69-0d0c189b44b7": {
    "_id": "4a19014e-16ef-4e51-9c69-0d0c189b44b7",
    "pclassid":"5c95397d4212cc40afeec915",
    "pfunction":{
      "signature":"sub(uint8)",
      "sources": {"python":"lambda a,b: a-b"},
      "graph":{},
      "gapi":{"constant": true,"inputs":[{"type":"uint8", "name":"a"},{"type":"uint8", "name":"b"}],"name":"io","outputs":[{"type":"uint8", "name":"sub"}],"payable": false,"stateMutability":"view","type":"function"}
    },
    "timestamp":"2019-03-22T14:38:36.112Z",
    "pclass": {
      "_id": "5c95397d4212cc40afeec915",
      "name": "sums",
      "type": "python",
      "deployment": "unknown"
    }
  }
}"""

context = json.loads(context_txt)


In [ ]:
graph1

In [ ]:
funcs = {}
getFunc(graph1['n'])

In [ ]:
instances

In [ ]:
enrich_graph (graph1)

In [ ]:
egraph = make_runtime (funcs, enrich_graph (graph1))
operate(egraph)

In [ ]:
context = json.loads(context_txt)

In [ ]:
%%javascript

console.log(d3);


In [ ]:
%load_ext py_d3

In [ ]:
from ipyleaflet import Map, Marker

center = (52.204793, 360.121558)

m = Map(center=center, zoom=15)

marker = Marker(location=center, draggable=True)
m.add_layer(marker);

display(m)

# Now that the marker is on the Map, you can drag it with your mouse,
# it will automatically update the `marker.location` attribute in Python

# You can also update the marker location from Python, that will update the
# marker location on the Map:
marker.location = (52.20476633165758, 360.12239456176763)


In [ ]:
marker.location


In [ ]:
from IPython.display import HTML


HTML('<script src="https://cdn.jsdelivr.net/npm/requirejs@2.3.6/bin/r.min.js"></script>')


In [ ]:
%%HTML
<script src="https://cdn.jsdelivr.net/npm/requirejs@2.3.6/bin/r.min.js"></script>

In [ ]:
%%d3

<g></g>

<script>
d3.select("g").text("Hello World");
</script>